# 「同性戀」怎麼了？

這次我們要看的是兩個語料中的「同性戀」參與哪些動作的角色、作為哪些動詞的論元，成為不同敘事中的主角。

以下一樣用 `jieba.posseg` 所提供的詞性標注功能，針對兩個語料當中，包含「同性戀」的句子進行詞性標註，並篩選出所有的動詞。

In [24]:
import nltk
import re
import json
import jieba
import jieba.posseg as pseg
# from ckip import CkipSegmenter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

jieba.set_dictionary('jieba_dict/dict.txt.big')
jieba.load_userdict('jieba_dict/user_dict.txt')
# segmenter = CkipSegmenter()

plt.rcParams['figure.figsize'] = [10, 5]
myfont = FontProperties(fname=r'/usr/local/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/msj.ttf',size=14)
sns.set(font=myfont.get_name())

%matplotlib inline

pd.set_option('display.max_rows', 2000)

LGBTFAMILY_PATH = "raw_content_pro.json"
LOVEFAMILY_PATH = "raw_content_anti.json"

lgbtfamily_raw_text = ""
lovefamily_raw_text = ""

with open(LGBTFAMILY_PATH) as f:
    lgbtfamily_raw_text = json.loads(f.read())

with open(LOVEFAMILY_PATH) as f:
    lovefamily_raw_text = json.loads(f.read())

Building prefix dict from /Users/koukantou/Desktop/Don/ntu_master/107-2-lab-meeting/nano-blog/jieba_dict/dict.txt.big ...
Loading model from cache /var/folders/vp/g4gpl3x91qqdn6_x0t8w4bbc0000gn/T/jieba.u336b009ae6d5715d70945d8db9630e20.cache
Loading model cost 2.965 seconds.
Prefix dict has been built succesfully.


In [25]:
# 前處理函式
def generate_line_sentence(all_text):  
    # 以句號分句
    all_text = re.split('[。，！？!?,]', all_text)
    
    # 去除所有非中文字元
#     all_text = [_get_chinese(s) for s in all_text]
    
    return all_text

In [26]:
# 接收語料(dict)與關鍵詞(string)，回傳語料當中關鍵詞出現的句子中，按照頻率排列的動詞表
def generate_verb_list(raw_text_list, word):
    a = [generate_line_sentence(article['content']) for article in raw_text_list]
    l = [sen for article in a for sen in article if word in sen]
    l = [list(pseg.cut(ll)) for ll in l]
    wordlist = [word for sen in l for (word, pos) in sen if pos.startswith('v')]
    freq_dict = {w:wordlist.count(w) for w in set(wordlist)}
    return sorted(freq_dict.items(), key=lambda kv: -kv[1])

# 接收語料(dict)與關鍵詞(string)，回傳語料中含有該關鍵詞的concordance dataframe
def generate_concordance_df(raw_text_list, word):
    a = [generate_line_sentence(article['content']) for article in raw_text_list]
    l = [sen.split(word) for article in a for sen in article if word in sen]
    df = pd.DataFrame({
    "left_context": [s[0] for s in l],
    "keyword": word,
    "right_context": [s[1] for s in l],
    })
    return df

## 伴侶盟中的同性戀

先看一下伴侶盟語料中含有「同性戀」的句子：

In [28]:
generate_concordance_df(lgbtfamily_raw_text, '同性戀')

,left_context,keyword,right_context
0,學生倘有,同性戀,傾向
1,而世界衛生組織早在1990年已將,同性戀,從心理疾病名單上剔除
2,指教青局副局長梁慧琪在本周四（16日）在塔石體育館出席一個活動後說：「如果（學生）展現,同性戀,傾向
3,如果學童或學生認為他們是,同性戀,
4,」被問到為何教青局認為,同性戀,學生需要臨床支援
5,「誰可以說他是否真的,同性戀,呢
6,當中有一些關於,同性戀,描述
7,不知道學校居然送,同性戀,學生到醫生處診治
8,世衛組織指1990年起,同性戀,已不再是疾病
9,因,同性戀,不屬心理疾病；惟學生正值青春期


再看伴侶盟語料中，經過`jieba`詞性標注後，所有含有「同性戀」一次句子中的動詞列表：

In [17]:
pd.DataFrame(generate_verb_list(lgbtfamily_raw_text, '同性戀'))

,0,1
0,是,85
1,有,30
2,認為,14
3,到,12
4,可能,12
5,保障,10
6,說,10
7,要,9
8,犯罪,9
9,研究,7


## 下福盟中的同性戀

先看下福盟語料中含有「同性戀」的句子：

In [29]:
generate_concordance_df(lovefamily_raw_text, '同性戀')

,left_context,keyword,right_context
0,由於近幾年來,同性戀,婚姻合法化的議題浮上檯面
1,關於,同性戀,情輔導案例
2,我不想把這樣的結果就直接指向,同性戀,情都是父親角色出問題所造成的原因
3,,同性戀,運動陣營若不訴諸殘暴的策略
4,大聲叫囂的同運惡棍和,同性戀,「佔領波士頓」組織早在2012年4月15日即曾干擾、淹沒茶黨在波士頓公園（Boston Co...
5,但不久之後…… 茶黨黨員及其家人子女聽見的卻是一群,同性戀,者高呼噁心的不倫戀口號和標語看板
6,左圖拿著「,同性戀,基督徒」標語的男子叫弋頓（Don Gorton）
7,他所屬的,同性戀,團體「加入衝擊」（Join the Impact）也在現場攪局
8,批評政府撥款贊助,同性戀,團體到各校散發文情並茂的情色刊物
9,大力促銷,同性戀,觀念


再看下福盟語料中，經過`jieba`
詞性標注後，所有含有「同性戀」一次句子中的動詞列表：

In [19]:
pd.DataFrame(generate_verb_list(lovefamily_raw_text, '同性戀'))

,0,1
0,是,1731
1,有,892
2,要,296
3,支持,291
4,生活,258
5,研究,251
6,教育,233
7,認為,215
8,保障,209
9,行為,202


## 觀察

### 治療同性戀
「同性戀」一詞的討論有一部份是關於從精神疾病中被除名，也因此「同性戀」這個詞的一個意思即屬於「前疾病」（套用下福盟喜歡使的「前同志」的構詞法：意指以前是同志，但後來不是），所以在兩個語料中，對於同性戀的討論有一部分是關於這是否是可治療或者需要被治療的。伴侶盟對下福盟的指控是下福盟的立場仍認為同性戀可以被治癒被恢復，而下福盟的於此的回應則是這是一個不實的標籤。

從這裡我們又回到上一篇關於「同性戀」一詞的語意場，其中之一包含了指涉類疾病的狀態（可能是由於歷史的因素），而「同志」一詞的語意比較沒有這樣的功能。


### 知名同性戀
在下福盟的語料中有兩個句子分別提及了「知名同性戀XXX(人名)」以及「知名同性戀樂團」。既然同性戀一詞作為與異性戀/雙性戀/無性戀...等性傾向相對的一種，那麼我們便可以試著造看看像是含有「國際知名異性戀XXX」或「台灣知名異性戀樂團」的句子，就會發現這樣的名詞詞組奇怪而有趣，當下福盟將人的一種性傾向提升為一種標籤，這是否反映著他們論述的策略之一：試圖放大一個人的性傾向，並以這樣的性傾向作為這群同性戀者的共同標籤。標籤作為標籤除了方便稱呼、命名分類之外，更不容忽視的是與此標籤綁定的性質，像是下福盟語料中的所謂「浸淫同性戀文化、性解放文化」，透過一個頓號，輕描淡寫地將同性戀與性解放並置：一邊是性傾向，一邊是社會運動。對於性解放究竟好或不好，可能因人而異，但當同性戀與性解放這兩個不那麼相關的概念被放在一起時，我們可以關注的是，為什麼這兩個詞語會被下福盟一起論及。

## 小結
至此，我花了四篇的篇幅討論了語料中「同性戀」與「同志」出現的句子，找出一些有趣的用法以及比較兩邊語料對於兩個近義詞的使用差異。我的確看到了這兩個詞語無論在使用脈絡上或語意上的差異，同志可以是一種自我認同、群體指涉、文化、生活方式，而同性戀可以是一種過去的精神疾病名、與異性戀雙性戀相對的某種戀、頭銜。

透過Concordance的方式，詞語的脈絡的確一目了然，不論是前後接的詞語還是整句話的事件動詞，但這只是對語料進行分析的其中一站，因為這樣的方式的一個缺點是將句子從他所在的文章中斷裂抽取出，無法更宏觀地看這個句子在整個文章中如何扮演串接整個段落的角色，也無法看出句子在段落中採取什麼樣修辭與論述的策略。

最近剛好在一堂課的讀本中讀到一句話：

> Computer-assisted text analysis [is] an activity best employed not in the service of a heightened critical objectivity, but as one that embraces the possibilities of that deepened subjectivity upon which critical insight depends.

這是 Stephen Ramsay 在其文章 "Toward an algorithmic criticism" (2003. Literary and Linguistic Computing, 18(2): 167-174)中所討論的，究竟電腦與演算法在言談分析和文本分析中扮演著什麼樣的角色。他認為演算法的嚴密與正確並不是分析的終點，客觀性與實證主義式的真理也不是所追求的目標，更重要的在於演算法是否讓分析者看見了原本看不見的，從而能針對文本做出更深刻的討論。（後來發現他有寫成一本書：[Reading Machines: Toward an Algorithmic Criticism](https://www.amazon.com/Reading-Machines-Algorithmic-Criticism-Humanities/dp/0252078209)）

在練習用不同的工具分析文本的過程中，好像真的看到了工具所看到的某種世界。